### Google Colab Setup

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [6]:
import os
import sys

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a WI2022 folder and put all the files under A5 folder, then "WI2022/A5"
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "CUB-200-2011"

GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))


# Add to sys so we can import .py files.

sys.path.append(GOOGLE_DRIVE_PATH) 

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'drive\\My Drive\\CUB-200-2011'

### Set the environment variable

In [1]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

### Load functions

In [4]:
from itertools import product
from solver import train_resnet_with_cub

### Hyper-parameters

In [3]:
# set hyper-parameters here
num_epochs = [10, 15, 20]
fine_tuning_lrs = [0.0001, 0.0005, 0.01]
output_lrs = [0.01, 0.02, 0.04, 0.06]

configurations = list(product(num_epochs, fine_tuning_lrs, output_lrs))

### Train

In [ ]:
best_accs = []

# train with the pretrained model
for config in configurations:
    curr_best_acc = train_resnet_with_cub(*config)
    best_accs.append(curr_best_acc)
    
# write the results into a txt file
with open('best_accuracy.txt', 'w') as f:
    for config, accuracy in zip(configurations, best_accs):
        f.write(f"Configuration: {config}, Accuracy: {accuracy}\n")